In [1]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy.optimize import curve_fit
import numpy.linalg as LA
import math
import struct
import diff_classifier.features as ft

from diff_classifier.utils import csv_to_pd
from diff_classifier.msd import nth_diff, msd_calc, all_msds


In [ ]:
frames = 2000
drift = 0
d = {'Frame': np.linspace(1, frames, frames),
     'Track_ID': np.ones(frames),
             'X': np.random.rand(frames)+drift*np.linspace(0, 1, frames),
             'Y': np.random.rand(frames)+drift*np.linspace(0, 1, frames)}
df = pd.DataFrame(data=d)
df = all_msds(df)
#df

In [ ]:
def calculate_features(df):

    # Skeleton of Trajectory features metadata table.
    # Builds entry for each unique Track ID.
    die = {'Track_ID': df.Track_ID.unique(),
          'alpha': df.Track_ID.unique(),
          'D_fit': df.Track_ID.unique(),
          'kurtosis': df.Track_ID.unique(),
          'asymmetry1': df.Track_ID.unique(),
          'asymmetry2': df.Track_ID.unique(),
          'asymmetry3': df.Track_ID.unique(),
          'AR': df.Track_ID.unique(),
          'elongation': df.Track_ID.unique(),
          'boundedness': df.Track_ID.unique(),
          'fractal_dim': df.Track_ID.unique(),
          'trappedness': df.Track_ID.unique(),
          'efficiency': df.Track_ID.unique(),
          'straightness': df.Track_ID.unique(),
          'MSD_ratio': df.Track_ID.unique()}
    di = pd.DataFrame(data=die)

    trackids = df.Track_ID.unique()
    partcount = trackids.shape[0]


    for particle in range(0, partcount):
        single_track = df.loc[df['Track_ID'] == trackids[particle]].sort_values(['Track_ID', 'Frame'],
                                                                                 ascending=[1, 1]).reset_index(drop=True)
        di['alpha'], di['D_fit'] = alpha_calc(single_track)
        di['kurtosis'] = kurtosis(single_track)
        l1, l2, di['asymmetry1'], di['asymmetry2'], di['asymmetry3'] = asymmetry(single_track)
        di['AR'], di['elongation'] = aspectratio(single_track)
        di['boundedness'], di['fractal_dim'], di['trappedness'] = boundedness(single_track)
        di['efficiency'], di['straightness'] = efficiency(single_track)
        di['MSD_ratio'] = msd_ratio(single_track, 2, single_track['Frame'][single_track.shape[0]-2])
    
    return di

In [ ]:
frames = 10
d = {'Frame': np.linspace(1, frames, frames),
             'X': np.linspace(1, frames, frames)+5,
             'Y': np.linspace(1, frames, frames)+3}
df = pd.DataFrame(data=d)
df['MSDs'], df['Gauss'] = msd_calc(df)

In [ ]:
frames = 10
d = {'Frame': np.linspace(1, frames, frames),
             'X': np.sin(np.linspace(1, frames, frames))+3,
             'Y': np.cos(np.linspace(1, frames, frames))+3}
df = pd.DataFrame(data=d)
df['MSDs'], df['Gauss'] = msd_calc(df)

In [ ]:
boundedness(df)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df['X'], df['Y'])
plt.show()

In [ ]:
df['Frame'][df['Frame'].shape[0]-1]

In [ ]:
def boundedness(track, framerate = 1):

    df = track
    length = df.shape[0]
    distance = np.zeros((length, length))

    for frame in range(0, length-1):
        distance[frame, 0:length-frame-1] = (np.sqrt(nth_diff(df['X'], frame+1)**2 + nth_diff(df['Y'], frame+1)**2).values)
    
    L = np.sum((np.sqrt(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2).values))
    r = np.max(distance)/2
    f = df['Frame'][df['Frame'].shape[0]-1]*framerate
    D = df['MSDs'][length-1]/(4*f)
    
    B = D*f/(r**2)
    Df = np.log(length-1)/np.log((length-1)*2*r/L)
    pf = 1 - np.exp(0.2048 - 0.25117*(D*f/(r**2)))

    return B, Df, pf

In [ ]:
def efficiency(track):

    df = track
    length = df.shape[0]
    num = (nth_diff(df['X'], length-1)**2 + nth_diff(df['Y'], length-1)**2)[0]
    num2 = np.sqrt(num)
    
    den = np.sum(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2)
    den2 = np.sum(np.sqrt(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2))
    
    eff = num/den
    strait = num2/den2
    return eff, strait

In [ ]:
def msd_ratio(track, n1=3, n2=100):

    df = track
    assert n1 < n2, "n1 must be less than n2"
    ratio = (df['MSDs'][n1]/df['MSDs'][n2]) - (df['Frame'][n1]/df['Frame'][n2])
    return ratio

In [ ]:
msd_ratio(df, 1, 9)

In [12]:
d = {'Frame': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
     'Track_ID': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
     'X': [5, 6, 7, 8, 9, 1, 2, 3, 4, 5],
     'Y': [6, 7, 8, 9, 10, 2, 3, 4, 5, 6]}
df = pd.DataFrame(data=d)
all_msds(df)

/home/ubuntu/source/diff-classifier/diff_classifier/msd.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['MSDs'][index1:index2], data['Gauss'][index1:index2] = msd_calc(single_track)
/home/ubuntu/source/diff-classifier/diff_classifier/msd.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['Frame'][index1:index2] = data['Frame'][index1:index2] - (data['Frame'][index1] - 1)


,Frame,Track_ID,X,Y,MSDs,Gauss
0,1,1,5,6,0.0,0.00
1,2,1,6,7,2.0,0.25
2,3,1,7,8,8.0,0.25
3,4,1,8,9,18.0,0.25
4,5,1,9,10,32.0,0.25
5,1,2,1,2,0.0,0.00
6,2,2,2,3,2.0,0.25
7,3,2,3,4,8.0,0.25
8,4,2,4,5,18.0,0.25
9,5,2,5,6,32.0,0.25


In [19]:
ft.calculate_features(df)

,AR,D_fit,MSD_ratio,Track_ID,alpha,asymmetry1,asymmetry2,asymmetry3,boundedness,efficiency,elongation,fractal_dim,kurtosis,straightness,trappedness
0,1.698414e+16,0.5,-0.35,1,2.0,1.0,1.110223e-16,0.693147,1.0,4.0,1.0,1.0,2.283058,1.0,0.045311
1,1.698414e+16,0.5,-0.35,2,2.0,1.0,1.110223e-16,0.693147,1.0,4.0,1.0,1.0,2.283058,1.0,0.045311


In [4]:
import pandas.util.testing as pdt

In [22]:
d = {'AR': np.ones(2)*1.698414e16,
     'D_fit': np.ones(2)*0.5,
     'MSD_ratio': np.ones(2)*-0.35,
     'Track_ID': [1, 2],
     'alpha': np.ones(2)*2.0,
     'asymmetry1': np.ones(2)*1.0,
     'asymmetry2': np.ones(2)*1.110223e-16,
     'asymmetry3': np.ones(2)*0.693147,
     'boundedness': np.ones(2)*1.0,
     'efficiency': np.ones(2)*4.0,
     'elongation': np.ones(2)*1.0,
     'fractal_dim': np.ones(2)*1.0,
     'kurtosis': np.ones(2)*2.283058,
     'straightness': np.ones(2)*1.0,
     'trappedness': np.ones(2)*0.04531133797073539}
dfi = pd.DataFrame(data=d)
dfi

,AR,D_fit,MSD_ratio,Track_ID,alpha,asymmetry1,asymmetry2,asymmetry3,boundedness,efficiency,elongation,fractal_dim,kurtosis,straightness,trappedness
0,1.698414e+16,0.5,-0.35,1,2.0,1.0,1.110223e-16,0.693147,1.0,4.0,1.0,1.0,2.283058,1.0,0.045311
1,1.698414e+16,0.5,-0.35,2,2.0,1.0,1.110223e-16,0.693147,1.0,4.0,1.0,1.0,2.283058,1.0,0.045311


In [24]:
pdt.assert_frame_equal(dfi, feat)